![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control
    
    Items.xml: Raw ingredient Items
    Ingredients.xml: Ingredients/Preps that goes into prep/products recipes
    Preps.xml: Preps that go into final product recipes
    Products.xml: Final Recipes/Menu Items
    Conversions.xml: Unit Conversion Information

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient *

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during ingredients processing process are ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process


## Import Libraries

In [1]:
#!pip3 install pdpipe

In [2]:
import numpy as np 
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os

import csv
from itertools import islice

In [3]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

In [4]:
import openpyxl

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

## Load Data

#### Data Path

In [6]:
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "oc", "*.oc"))

In [7]:
filepath_list

['/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1121.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1346.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03122021_0915.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1224.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1250.oc']

#### Load Items List

In [8]:
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/items.xml')
    xroot = xtree.getroot()
    for item in xtree.iterfind('Item'):
        ItemId.append(item.attrib['id'])
        Description.append(item.findtext('Description'))
        CaseQty.append(item.findtext('CaseQty'))
        CaseUOM.append(item.findtext('CaseUOM'))
        PakQty.append(item.findtext('PakQty'))
        PakUOM.append(item.findtext('PakUOM'))
        InventoryGroup.append(item.findtext('InventoryGroup'))

        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    ).drop_duplicates()

Items.reset_index(drop=True, inplace=True)

In [9]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4472,AVOCADO MX,20.000,CT,1.000,CT,PRODUCE
1,I-4973,AVOCADO PULP CHUNKY,12.000,bag,454.000,g,PRODUCE
2,I-27410,BACON 3MM NATURALLY SMKD,5.000,Kg,1.000,Kg,MEAT
3,I-1286,BAGEL WHOLE WHEAT (6 PACK),6.000,CT,1.000,CT,BREAD
4,I-3130,BASE BEEF CONC LIQ G/FREE,4.000,pak,946.000,ml,FOOD - GROCERY


In [10]:
Items.shape

(390, 7)

In [11]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

#### Load Ingredient List

In [12]:
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Ingredients.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Ingredient'):
        IngredientId.append(x.attrib['ingredient'])
        Conversion.append(x.attrib['conversion'])
        InvFactor.append(x.attrib['invFactor'])
        Qty.append(x.attrib['qty'])
        Recipe.append(x.attrib['recipe'])
        Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()

Ingredients.reset_index(drop=True, inplace=True)

In [13]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3388,1.000,L,1.00000000,0.3058,P-10496
1,I-4660,2.270,Kg,2.20462000,0.6942,P-10496
2,I-4679,1.000,BUNCH,1.00000000,0.0063,P-18318
3,I-4793,10.000,Kg,2.20462000,1.2048,P-18746
4,I-3643,225.000,g,0.00100000,0.1837,P-18907


In [14]:
Ingredients.shape

(2198, 6)

In [15]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [16]:
path = os.path.join(os.getcwd(), "data", "processed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

#### Load Preps List

In [17]:
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Preps.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Prep'):
        PrepId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        PakQty.append(x.findtext('PakQty'))
        PakUOM.append(x.findtext('PakUOM'))
        InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()

Preps.reset_index(drop=True, inplace=True)

In [18]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-56398,BATCH| Guacamole AR,2.500,Kg,PREP
1,P-26333,BATCH|Citrus Herb Aioli,3.000,L,PREP
2,P-35713,BLACKENED|Chicken,185.000,ea,PREP
3,P-51579,BREADED|Chicken|Cooked,1.600,Kg,
4,P-50831,BREADED|Chicken|Karaage Thigh,4.500,Kg,


In [19]:
Preps.shape

(427, 5)

In [20]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

#### Load Product List

In [21]:
ProdId = []
Description = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Products.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Prod'):
        ProdId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description}).drop_duplicates()

Products.reset_index(drop=True, inplace=True)

In [22]:
Products.head()

,ProdId,Description
0,R-36666,FF|Bowl|Five Alarm
1,R-36568,FF|Pasta|Spinach Avocado
2,R-50362,FLX|Bowl|Karaage Chicken
3,R-50360,FLX|Bowl|Salmon
4,R-50359,FLX|Bowl|Tuna Albacore


In [23]:
Products.shape

(103, 2)

In [24]:
path = os.path.join(os.getcwd(), "data", "processed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

#### Load Conversion List

In [25]:
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Conversions.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Conversion'):
        ConversionId.append(x.attrib['id'])
        Multiplier.append(x.attrib['multiplier'])
        ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
        ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
        ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
        ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [26]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L


In [27]:
Conversions.shape

(240, 6)

In [28]:
path = os.path.join(os.getcwd(), "data", "processed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

#### Load Emission Factors

In [29]:
cfc_filename = os.path.join(os.getcwd(), "data", "raw", "cool_food_ghg.csv")
cfc = pd.read_csv(cfc_filename)
cfc.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [30]:
cfc.shape

(54, 3)

## Data Cleaning

#### Unit Conversion

In [31]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

for index in Preps.index:
    if Preps.loc[index,'PakUOM'] == 'Kg':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*1000
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'lb':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*453.592
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'oz':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*28.3495
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'L':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*1000
        Preps.loc[index,'StdUom'] = 'ml'
    else: 
        Preps.loc[index,'StdQty'] = Preps.loc[index,'PakQty']
        Preps.loc[index,'StdUom'] = Preps.loc[index,'PakUOM']
        
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-56398,BATCH| Guacamole AR,2.500,Kg,PREP,2500,g
1,P-26333,BATCH|Citrus Herb Aioli,3.000,L,PREP,3000,ml
2,P-35713,BLACKENED|Chicken,185.000,ea,PREP,185.000,ea
3,P-51579,BREADED|Chicken|Cooked,1.600,Kg,,1600,g
4,P-50831,BREADED|Chicken|Karaage Thigh,4.500,Kg,,4500,g


In [32]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

## Data Analysis

#### Mapping Items to GHG Factors

In [33]:
# Load Items List with assigned Cetegory ID
items = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv"))

In [34]:
df1 = pd.DataFrame(items)
df2 = pd.DataFrame(cfc)

In [35]:
df1.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT
1,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
4,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT


In [36]:
df2.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [37]:
mapping = pd.merge(df1, df2.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

In [38]:
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0

In [39]:
mapping.loc[337,'Category ID'].dtype

dtype('float64')

In [40]:
mapping.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,1.0,beef & buffalo meat,41.3463
1,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,1.0,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,1.0,beef & buffalo meat,41.3463
3,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,1.0,beef & buffalo meat,41.3463
4,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT,1.0,beef & buffalo meat,41.3463


In [41]:
mapping.shape

(390, 11)

In [42]:
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Category ID                                               float64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

#### GHG Factors Calculation for Preps

In [43]:
Preps['GHG Emission/g'] = np.nan

In [44]:
for index, row in Preps.iterrows():
    ghgf = 0
    Preps.loc[index, 'GHG Emission/g'] = ghgf

In [45]:
def get_items_prep_ghgf(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghgf = Preps.loc[index, 'GHG Emission/g']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            if ingres.loc[idx,'Uom'] in ['g', 'ml', 'Kg']:
                weight = float(ingres.loc[idx,'Conversion'])*float(ingres.loc[idx,'InvFactor'])
                ghgf += weight*float(ghge)
    Preps.loc[index, 'GHG Emission/g'] = float(ghgf)

In [46]:
for index, row in Preps.iterrows():
    get_items_prep_ghgf(index , row)

In [47]:
Preps#.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission/g
0,P-56398,BATCH| Guacamole AR,2.500,Kg,PREP,2500,g,0.000000
1,P-26333,BATCH|Citrus Herb Aioli,3.000,L,PREP,3000,ml,1.666188
2,P-35713,BLACKENED|Chicken,185.000,ea,PREP,185.000,ea,0.546870
3,P-51579,BREADED|Chicken|Cooked,1.600,Kg,,1600,g,3.299700
4,P-50831,BREADED|Chicken|Karaage Thigh,4.500,Kg,,4500,g,61.755573
5,P-45062,BREADING|Chick Wings|GFree,7.300,Kg,PREP,7300,g,573.415827
6,P-50535,BRINE|Pork|Liquid,43.000,L,,43000,ml,501.626176
7,P-51131,BRINED|Pork Butt|Marinated,8.000,Kg,,8000,g,9.339925
8,P-50500,CHIFFONADE|Basil,190.000,g,,190.000,g,0.001167
9,P-24750,CHOPPED|Cilantro,0.500,Kg,,500,g,0.000000


### GHG Calculation for Products

In [48]:
Products['GHG Emission (g)'] = np.nan

In [49]:
for index, row in Products.iterrows():
    ghg = 0
    Products.loc[index, 'GHG Emission (g)'] = ghg

In [50]:
ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[1,'ProdId']]
ingres

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
562,I-2141,10.000,g,0.00100000,1.0000,R-36568
563,I-3387,15.000,ml,0.00100000,1.0000,R-36568
564,I-4463,10.000,g,0.00220462,1.0000,R-36568
565,P-25993,0.500,g,1.00000000,1.0000,R-36568
566,P-26216,180.000,g,0.00100000,1.0000,R-36568
567,P-36566,100.000,ml,0.00100000,1.0000,R-36568
568,P-50500,2.000,g,1.00000000,1.0000,R-36568
569,P-50966,40.000,g,0.00100000,1.0000,R-36568


In [51]:
ghg = 0
for idx, row in ingres.iterrows():
    ingre = ingres.loc[idx,'IngredientId']
    if ingre.startswith('I'):
        ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
        if (ingres.loc[idx,'Uom'] == 'g') or (ingres.loc[idx,'Uom'] == 'ml'):
            qty = float(ingres.loc[idx,'Qty'])
            ghg += qty*float(ghge)   
        print(ghge)
        print(ghg)
ghg

36    8.9104
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
89.10399999999998
264    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
137.70549999999997
151    0.622
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
143.92549999999997


143.92549999999997

In [52]:
def get_items_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            if (ingres.loc[idx,'Uom'] == 'g') or (ingres.loc[idx,'Uom'] == 'ml'):
                qty = float(ingres.loc[idx,'Qty'])
                ghg += qty*float(ghge)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)

In [53]:
#def get_preps_ghge(index, row):
    

In [54]:
for index, row in Products.iterrows():
    get_items_ghge(index , row)

In [55]:
Products

,ProdId,Description,GHG Emission (g)
0,R-36666,FF|Bowl|Five Alarm,0.00000
1,R-36568,FF|Pasta|Spinach Avocado,143.92550
2,R-50362,FLX|Bowl|Karaage Chicken,17.22400
3,R-50360,FLX|Bowl|Salmon,0.00000
4,R-50359,FLX|Bowl|Tuna Albacore,45.05360
5,R-44362,FLX|Fried Cauliflower Bowl|Med,0.00000
6,R-44319,FLX|Side Quinoa Tabouleh Salad,0.00000
7,R-44583,FLX|Taco|Mojo pork,0.00000
8,R-30406,G&G|Croissant|Multigrain|OK,0.00000
9,R-32750,GLB|Chicken Adobo,0.00000


In [56]:
path = os.path.join(os.getcwd(), "data", "final", "Recipes GHG.csv")
Products.to_csv(path, index = False, header = True)